Cobra documentation: https://cobrapy.readthedocs.io/en/latest/index.html



In case Cobra doesn't run do the following:

* Go to the Files list and open a terminal (`terminal.term`)
* Type: `pip3 install --upgrade python-libsbml`
* Type: `pip3 install --upgrade sympy`
* Type: `pip3 install --upgrade cobra`
* Go to Settings and click on `Restart project`



In [3]:
import cobra
import matplotlib.pyplot as plt
import random
random.seed(42)

## Exercise: using a metabolic model of <i>Chlamydomonas reinhardtii</i>.

- Read the model, which is available in the file "iRC1080\-Chapman3.xml" \(you can ignore any warning messages\).



In [4]:
import cobra
chlamyNet = "iRC1080-Chapman3.xml"
model = cobra.io.read_sbml_model(chlamyNet)

'' is not a valid SBML 'SId'.


No objective coefficients in model. Unclear what should be optimized


* Print the number of reactions, metabolites and genes.

In [7]:
print("The number of reactions:", len(model.reactions))
print("The number of metabolites:", len(model.metabolites))
print("The number of genes:", len(model.genes))

The number of reactions: 2191
The number of metabolites: 1706
The number of genes: 1086


* Find all occurrences of fumarate in the model and print their IDs.
* In which cellular compartments is fumarate present?



In [16]:
for m in model.metabolites:
    if "Fumarate" in m.name:
        print(m.id, ",", m.name, ",", m.compartment)

fum_c , Fumarate , c
fum_h , Fumarate , h
fum_m , Fumarate , m
c


* Print the chemical formula and charge of fumarate.

In [20]:
fumarate_c = model.metabolites.get_by_id("fum_c")
fumarate_h = model.metabolites.get_by_id("fum_h")
fumarate_m = model.metabolites.get_by_id("fum_m")

print("Chemical Formula:", fumarate_h.formula)
print("Charge:", fumarate_h.charge)


Chemical Formula: C4H2O4
Charge: 0


* List all reactions containing fumarate.

In [34]:
for r in fumarate_c.reactions:
    print(r.id, r.name, r.reaction,"Flux:",r.flux)
    
for r in fumarate_h.reactions:
    print(r.id, r.name, r.reaction,"Flux:",r.flux)
    
for r in fumarate_m.reactions:
    print(r.id, r.name, r.reaction,"Flux:",r.flux)

ARGSL argininosuccinate lyase argsuc_c <=> arg_DASH_L_c + fum_c Flux: 0.9247425541326856
FAA fumarylacetoacetase 4fumacac_c + h2o_c --> acac_c + fum_c + h_c Flux: 0.0
AAL_LPAREN_fum_RPAREN_ N6-(1,2-dicarboxyethyl)AMP AMP-lyase (fumarate-forming) dcamp_c --> amp_c + fum_c Flux: 0.3817652896001283
SUCFUMthr succinate-fumarate antiport, chloroplast fum_h + succ_c <=> fum_c + succ_h Flux: 0.0
SUCFUMtmr Succinate/fumarate antiporter, mitochondrial fum_m + succ_c <=> fum_c + succ_m Flux: -2.3462514225021764
AIAL 1-(5'-Phosphoribosyl)-5-amino-4-(N-succinocarboxamide)-imidazole AMP-lyase 25aics_c <=> aicar_c + fum_c Flux: 1.039743578769362
SUCDHh succinate dehydrogenase (chloroplast) fad_h + succ_h --> fadh2_h + fum_h Flux: 0.0
SUCFUMthr succinate-fumarate antiport, chloroplast fum_h + succ_c <=> fum_c + succ_h Flux: 0.0
SUCDH_LPAREN_q8_RPAREN_m succinate dehydrogenase (ubiquinone) q8_m + succ_m <=> fum_m + q8h2_m Flux: -2.989923024832095e-14
FUMm fumarate hydratase fum_m + h2o_m <=> mal_DASH_

* Set the biomass objective to heterotrophic growth.
* Run FBA and print the objective value (biomass production).

In [24]:
biomass = model.reactions.get_by_id("Biomass_Chlamy_auto")
biomass.objective_coefficient = 1

solution = model.optimize()
print("FBA solution:", solution.objective_value)

FBA solution: 6.156851475933877


* Print the flux of all reactions containing fumarate.

In [38]:
#print(fumarate_c.summary())
#print(fumarate_h.summary())
#print(fumarate_m.summary())

for r in fumarate_c.reactions:
    print(r.flux)
    
for r in fumarate_h.reactions:
    print(r.flux)
    
for r in fumarate_m.reactions:
    print(r.flux)

0.9247425541326856
0.0
0.3817652896001283
0.0
-2.3462514225021764
1.039743578769362
0.0
0.0
-2.989923024832095e-14
2.3462514225021764
-2.3462514225021764
2.989923024832095e-14


* Find the reaction that imports extracellular acetate into the cell.

In [75]:
acetate = model.metabolites.get_by_id("ac_e")
print(acetate.summary())

ac_e
====
Formula: C2H3O2

Producing Reactions
-------------------
Percent Flux Reaction    Definition
100.00%   20      ACt ac_e <-- ac_c

Consuming Reactions
-------------------
Percent Flux               Reaction Definition
100.00%  -20 EX_ac_LPAREN_e_RPAREN_  ac_e <=> 


* Set the acetate import flux to 10.
* Run FBA and print the objective value.

In [39]:
model.reactions.get_by_id("ACt").lower_bound = -10
model.reactions.get_by_id("ACt").upper_bound = 0

solution = model.optimize()
print("FBA solution:", solution.objective_value)


FBA solution: 6.156851475934288


* Print the flux of all reactions containing fumarate and compare to the previous solution.

In [44]:
print(fumarate_c.summary())
print(fumarate_h.summary())
print(fumarate_m.summary())

for r in fumarate_c.reactions:
    print(r.flux)
    
for r in fumarate_h.reactions:
    print(r.flux)
    
for r in fumarate_m.reactions:
    print(r.flux)

fum_c
=====
Formula: C4H2O4

Producing Reactions
-------------------
Percent   Flux               Reaction                        Definition
 16.27% 0.3818 AAL_LPAREN_fum_RPAREN_         dcamp_c --> amp_c + fum_c
 44.32%   1.04                   AIAL      25aics_c <=> aicar_c + fum_c
 39.41% 0.9247                  ARGSL argsuc_c <=> arg_DASH_L_c + fum_c

Consuming Reactions
-------------------
Percent   Flux  Reaction                        Definition
100.00% -2.346 SUCFUMtmr fum_m + succ_c <=> fum_c + succ_m
fum_h
=====
Formula: C4H2O4

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []


fum_m
=====
Formula: C4H2O4

Producing Reactions
-------------------
Percent  Flux  Reaction                        Definition
100.00% 2.346 SUCFUMtmr fum_m + succ_c <=> fum_c + succ_m

Consuming Reactions
-------------------
Percent   Flux Reaction                     Definition
100.00% -2.346     FUMm fum_m + h2o_m <=> mal_DASH_L_m
0.9247425541326856
0.0
0.3817652896001283
0.0
-2.3462514225021764
1.039743578769362
0.0
0.0
-2.989923024832095e-14
2.3462514225021764
-2.3462514225021764
2.989923024832095e-14


* Run a series of FBA simulations with the acetate uptake flux ranging from 0 to 20.
* Plot a line graph showing how the flux of the argininosuccinate lyase reaction varies depending on the acetate uptake.

In [86]:
import matplotlib.pyplot as plt

exac = model.reactions.get_by_id("EX_ac_LPAREN_e_RPAREN_")

lyase_flux = list()

for i in range(0,21):
    exac.bounds=(-i,-i)
    solution = model.optimize()
    print(solution.fluxes["ARGSL"])
    print(solution.objective_value)
    lyase_flux.append(solution.fluxes["ARGSL"])
    print(lyase_flux)


x = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

plt.plot(x,lyase_flux)

plt.xlabel("Acetate uptake flux")
plt.ylabel("Argininosuccinate lyase flux")

plt.show()




0.0
0.0
[0.0]
0.0
0.0
[0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
